source: https://medium.com/@hernandezgalaviz/using-valhalla-map-matching-to-get-route-and-travelled-distance-from-raw-gps-points-4ea6b1c88a4c

In [2]:
import folium
import pandas as pd
import geopandas as gpd
import json
import requests
from shapely.geometry.linestring import LineString
from pyproj import Geod

In [3]:
#decode an encoded string
def decode(encoded):
  inv = 1.0 / 1e6
  decoded = []
  previous = [0,0]
  i = 0
  #for each byte
  while i < len(encoded):
    #for each coord (lat, lon)
    ll = [0,0]
    for j in [0, 1]:
      shift = 0
      byte = 0x20
      #keep decoding bytes until you have this coord
      while byte >= 0x20:
        byte = ord(encoded[i]) - 63
        i += 1
        ll[j] |= (byte & 0x1f) << shift
        shift += 5
      #get the final value adding the previous offset and remember it for the next
      ll[j] = previous[j] + (~(ll[j] >> 1) if ll[j] & 1 else (ll[j] >> 1))
      previous[j] = ll[j]
    #scale by the precision and chop off long coords also flip the positions so
    #its the far more standard lon,lat instead of lat,lon
    decoded.append([float('%.6f' % (ll[1] * inv)), float('%.6f' % (ll[0] * inv))])
  #hand back the list of coordinates
  return decoded

In [19]:

#%% READ & FORMAT GPS INFO
geojson_file = '../data/test_valhalla.geojson'
gdf_rawGPS_linestring = gpd.read_file(geojson_file)
gdf_rawGPS_points_temp = gdf_rawGPS_linestring.apply(lambda x: [y for y in x['geometry'].coords], axis=1)
gdf_rawGPS_points = gpd.GeoDataFrame(geometry=gpd.points_from_xy([a_tuple[0] for a_tuple in gdf_rawGPS_points_temp[0]], [a_tuple[1] for a_tuple in gdf_rawGPS_points_temp[0]]), crs=4326)
df_rawGPS_points = pd.DataFrame(list(zip([a_tuple[0] for a_tuple in gdf_rawGPS_points_temp[0]],[a_tuple[1] for a_tuple in gdf_rawGPS_points_temp[0]])) , columns=['lon', 'lat'])
gdf_rawGPS = gpd.GeoDataFrame(pd.concat([gdf_rawGPS_linestring, gdf_rawGPS_points], ignore_index=True))

In [60]:
df_rawGPS_points

,lon,lat
0,-2.572997,51.514453
1,-2.573174,51.514233
2,-2.573315,51.514035
3,-2.573209,51.513838
4,-2.573633,51.513750
...,...,...
89,-2.575944,51.506879
90,-2.575654,51.506907
91,-2.575601,51.507002
92,-2.575654,51.507080


In [20]:

#%% VALHALLA REQUEST
meili_coordinates = df_rawGPS_points.to_json(orient='records')
meili_head = '{"shape":'
meili_tail = ""","search_radius": 300, "shape_match":"map_snap", "costing":"auto", "format":"osrm"}"""
meili_request_body = meili_head + meili_coordinates + meili_tail
url = "http://localhost:8002/trace_route"
headers = {'Content-type': 'application/json'}
data = str(meili_request_body)
r = requests.post(url, data=data, headers=headers)

In [13]:
meili_coordinates

'[{"lon":-2.59716,"lat":51.454426},{"lon":-2.59743,"lat":51.453561},{"lon":-2.597381,"lat":51.452118},{"lon":-2.597033,"lat":51.452396},{"lon":-2.597095,"lat":51.452443},{"lon":-2.597191,"lat":51.452483},{"lon":-2.597175,"lat":51.452483},{"lon":-2.597393,"lat":51.452726},{"lon":-2.597063,"lat":51.453283},{"lon":-2.596875,"lat":51.45385},{"lon":-2.596238,"lat":51.455451},{"lon":-2.596093,"lat":51.455948},{"lon":-2.595773,"lat":51.456466},{"lon":-2.59287,"lat":51.458138},{"lon":-2.591683,"lat":51.458578},{"lon":-2.591253,"lat":51.458933},{"lon":-2.591265,"lat":51.458916},{"lon":-2.591066,"lat":51.458971},{"lon":-2.591123,"lat":51.459026},{"lon":-2.591138,"lat":51.459698},{"lon":-2.590823,"lat":51.46079},{"lon":-2.589983,"lat":51.462956},{"lon":-2.58964,"lat":51.464235},{"lon":-2.589618,"lat":51.464505},{"lon":-2.590173,"lat":51.465445},{"lon":-2.590376,"lat":51.465836},{"lon":-2.591405,"lat":51.467856},{"lon":-2.591693,"lat":51.468416},{"lon":-2.592093,"lat":51.46887},{"lon":-2.592945,"l

In [21]:

#%% READ & FORMAT VALHALLA RESPONSE
if r.status_code == 200:
    response_text = json.loads(r.text)
search_1 = response_text.get('matchings')
search_2 = dict(search_1[0])
polyline6 = search_2.get('geometry')
search_3 = response_text.get('tracepoints')

lst_MapMatchingRoute = LineString(decode(polyline6))
gdf_MapMatchingRoute_linestring = gpd.GeoDataFrame(geometry=[lst_MapMatchingRoute], crs=4326)
# Convert from geopanda object to a list of coordinates for each row
gdf_MapMatchingRoute_points_temp = gdf_MapMatchingRoute_linestring.apply(lambda x: [y for y in x['geometry'].coords], axis=1)
# Take the first extracted row and etract individual coordinates
gdf_MapMatchingRoute_points = gpd.GeoDataFrame(geometry=gpd.points_from_xy([a_tuple[0] for a_tuple in gdf_MapMatchingRoute_points_temp[0]], [a_tuple[1] for a_tuple in gdf_MapMatchingRoute_points_temp[0]]), crs=4326)
gdf_MapMatchingRoute = gpd.GeoDataFrame(pd.concat([gdf_MapMatchingRoute_linestring, gdf_MapMatchingRoute_points], ignore_index=True))
df_mapmatchedGPS_points = pd.DataFrame(list([d['location'] for d in search_3 if 'location' in d]) , columns=['lon', 'lat'])
gdf_mapmatchedGPS_points = gpd.GeoDataFrame(geometry=gpd.points_from_xy(df_mapmatchedGPS_points['lon'], df_mapmatchedGPS_points['lat']), crs=4326)

In [62]:
df_mapmatchedGPS_points

,lon,lat
0,-2.572823,51.514364
1,-2.573068,51.514179
2,-2.573282,51.514018
3,-2.573387,51.513948
4,-2.573669,51.513772
...,...,...
89,-2.575919,51.506943
90,-2.575645,51.506903
91,-2.575603,51.507002
92,-2.575597,51.507016


In [59]:
pd.DataFrame([d['location'] for d in search_3 if 'location' in d], columns=['lat','lon'])

,lat,lon
0,-2.572823,51.514364
1,-2.573068,51.514179
2,-2.573282,51.514018
3,-2.573387,51.513948
4,-2.573669,51.513772
...,...,...
89,-2.575919,51.506943
90,-2.575645,51.506903
91,-2.575603,51.507002
92,-2.575597,51.507016


In [65]:
gdf_MapMatchingRoute

,geometry
0,"LINESTRING (-2.57282 51.51436, -2.57332 51.513..."
1,POINT (-2.57282 51.51436)
2,POINT (-2.57332 51.51399)
3,POINT (-2.57372 51.51374)
4,POINT (-2.57398 51.51351)
...,...
69,POINT (-2.57640 51.50764)
70,POINT (-2.57659 51.50704)
71,POINT (-2.57644 51.50702)
72,POINT (-2.57565 51.50690)


In [22]:

#%% RAW & MAP-MATCHING ROUTES - DRAW MAP
m = folium.Map([51.49856419519174, -2.548053188420676], tiles='cartodbdark_matter', zoom_start=15)
# m = folium.Map([51.49856419519174, -2.548053188420676], tiles='OpenStreetMap', zoom_start=14)
folium.GeoJson(gdf_rawGPS, style_function=lambda x:{'color': 'red'}, marker=folium.CircleMarker(radius=4, weight=0, fill_color='red', fill_opacity=1), name='rawGPS_points').add_to(m)
folium.GeoJson(gdf_mapmatchedGPS_points, marker=folium.CircleMarker(radius=4, weight=0, fill_color='white', fill_opacity=1), name='MapMatching_rawGPS_points').add_to(m)
folium.GeoJson(gdf_MapMatchingRoute, style_function=lambda x:{'color': 'green'}, marker=folium.CircleMarker(radius=4, weight=0, fill_color='green', fill_opacity=1), name='MapMatching_Route').add_to(m)
folium.LayerControl(position='topright', collapsed=False).add_to(m)
# m.save('mapmatching.html')
m

In [23]:

#%% RAW & MAP-MATCHING ROUTES - CALCULATE DISTANCE
geod = Geod(ellps="WGS84")
rawGPS_linestring_distance = geod.geometry_length(gdf_rawGPS_linestring['geometry'][0])
MapMatchingRoute_linestring_distance = geod.geometry_length(gdf_MapMatchingRoute_linestring['geometry'][0])
print('rawGPS_linestring_distance = ', f"{rawGPS_linestring_distance:,.0f}")
print('MapMatchingRoute_linestring_distance = ', f"{MapMatchingRoute_linestring_distance:,.0f}")

rawGPS_linestring_distance =  1,397
MapMatchingRoute_linestring_distance =  1,182


## Running on Preprocessed Files

In [52]:
pkl_file = '../data/processed/trips_matched/p_70_20240725.pkl'
pkl_file = pd.read_pickle(pkl_file)
trip_id = pkl_file['TripId'].unique().tolist()[2]

pkl_data = pkl_file[pkl_file['TripId']==trip_id]

df_rawGPS_points = pd.DataFrame()
df_rawGPS_points['lon'] = pkl_data['Longitude']
df_rawGPS_points['lat'] = pkl_data['Latitude']

df_mapmatchedGPS_points = pd.DataFrame()
df_mapmatchedGPS_points['lon'] = pkl_data['LongitudeMatched']
df_mapmatchedGPS_points['lat'] = pkl_data['LatitudeMatched']

gdf_rawGPS = gpd.GeoDataFrame(geometry=[LineString([*zip(df_rawGPS_points['lon'], df_rawGPS_points['lat'])])], crs=4326)
gdf_mapmatchedGPS_points = gpd.GeoDataFrame(geometry=[LineString([*zip(df_mapmatchedGPS_points['lon'], df_mapmatchedGPS_points['lat'])])], crs=4326)
gdf_MapMatchingRoute = gpd.GeoDataFrame(geometry=[eval(pkl_data.iloc[0]['EstimatedRoute'])], crs=4326)

In [63]:

#%% RAW & MAP-MATCHING ROUTES - DRAW MAP
f = folium.Figure(800, 800)
m = folium.Map([51.49856419519174, -2.548053188420676], tiles='cartodbdark_matter', zoom_start=14).add_to(f)
# m = folium.Map([51.49856419519174, -2.548053188420676], tiles='OpenStreetMap', zoom_start=14)
folium.GeoJson(gdf_MapMatchingRoute, style_function=lambda x:{'color': 'green'}, marker=folium.CircleMarker(radius=4, weight=0, fill_color='green', fill_opacity=1), name='Map Macthed (BLUE)').add_to(m)
folium.GeoJson(gdf_rawGPS, style_function=lambda x:{'color': 'red', 'opacity': 0.8}, marker=folium.CircleMarker(radius=4, weight=0, fill_color='red', fill_opacity=1), name='Raw Linestring (RED)').add_to(m)
# folium.GeoJson(gdf_mapmatchedGPS_points, marker=folium.CircleMarker(radius=4, weight=0, fill_color='white', fill_opacity=1), name='MapMatching_rawGPS_points').add_to(m)
folium.LayerControl(position='topright', collapsed=False).add_to(m)
# m.save('mapmatching.html')


#%% RAW & MAP-MATCHING ROUTES - CALCULATE DISTANCE
geod = Geod(ellps="WGS84")
rawGPS_linestring_distance = geod.geometry_length(gdf_rawGPS['geometry'][0])
MapMatchingRoute_linestring_distance = geod.geometry_length(gdf_MapMatchingRoute['geometry'][0])
print(f"Line {pkl_data.iloc[0]['LineRef']} - {pkl_data.iloc[0]['DirectionRef']} - {pkl_data.iloc[0]['OriginAimedDepartureTime']}")
print('Raw GPS Distance Travelled = ', f"{rawGPS_linestring_distance:,.0f}")
print('Map-Matched Distance Travelled = ', f"{MapMatchingRoute_linestring_distance:,.0f}")

m

Line 70 - outbound - 2024-07-25 01:55:00+00:00
Raw GPS Distance Travelled =  9,551
Map-Matched Distance Travelled =  10,138
